In [1]:
setwd(paste0(Sys.getenv('ROOT'), '/R'))
getwd()

[1] "/mnt/hdd/develop/kaggle/house-prices/R/R"

In [30]:
source('main.R')

data <- kaggle.house$loadData()

df.combined <- kaggle.house$getCombinedDataset(data$train, data$test) %>% (kaggle.house$na$fixAll)

df.training.raw <- 
    df.combined %>% 
    filter(dataSource == "train") %>% 
    mutate(sale_price_log = log(SalePrice)) %>%
    select(-dataSource, -SalePrice)
    
df.testing.raw  <- 
    df.combined %>% 
    filter(dataSource == "test") %>% 
    select(-dataSource, -SalePrice)

tran.res <- kaggle.house$trans$doItAll(df.training.raw, df.testing.raw)
df.training <- tran.res$df.training
df.testing <- tran.res$df.testing

# Should be only numeric columns
stopifnot(0 == df.training %>% purrr::map(function (col) { !is.numeric(col) }) %>% unlist %>% sum)
stopifnot(0 == df.testing %>% purrr::map(function (col) { !is.numeric(col) }) %>% unlist %>% sum)

df.training2 <- 
    df.training %>% mutate(
        BsmtFinSF2=log(BsmtFinSF2 + 1),
        MiscVal=log(MiscVal + 1),
        LotArea=log(LotArea + 1),
        LowQualFinSF=log(LowQualFinSF + 1),
        OpenPorchSF=log(OpenPorchSF + 1),
        EnclosedPorch=log(EnclosedPorch + 1)
    )
df.testing2 <- 
    df.testing %>% mutate(
        BsmtFinSF2=log(BsmtFinSF2 + 1),
        MiscVal=log(MiscVal + 1),
        LotArea=log(LotArea + 1),
        LowQualFinSF=log(LowQualFinSF + 1),
        OpenPorchSF=log(OpenPorchSF + 1),
        EnclosedPorch=log(EnclosedPorch + 1)
    )

In [32]:
partition <- createDataPartition(y=df.training$sale_price_log, p=.5, list=F)

df.training.train <- df.training[partition,]
df.training.test <- df.training[-partition,]

model_1 = lm(sale_price_log ~ ., data=df.training.train)
summary(model_1)

df.training2.train <- df.training2[partition,]
df.training2.test <- df.training2[-partition,]

model_2 = lm(sale_price_log ~ ., data=df.training2.train)
summary(model_2)


Call:
lm(formula = sale_price_log ~ ., data = df.training.train)

Residuals:
     Min       1Q   Median       3Q      Max 
-0.97293 -0.05842  0.00794  0.06647  0.40379 

Coefficients: (2 not defined because of singularities)
                         Estimate Std. Error t value Pr(>|t|)    
(Intercept)             6.141e+00  8.540e+00   0.719 0.472366    
LotFrontage             1.654e-04  1.670e-04   0.990 0.322561    
LotArea                 1.381e-06  6.959e-07   1.984 0.047629 *  
OverallQual             4.964e-02  7.134e-03   6.958 8.40e-12 ***
OverallCond             5.099e-02  6.396e-03   7.972 6.99e-15 ***
YearBuilt               1.086e-03  4.864e-04   2.234 0.025845 *  
YearRemodAdd            7.126e-05  4.059e-04   0.176 0.860708    
MasVnrArea             -2.125e-05  3.838e-05  -0.554 0.579866    
BsmtFinSF1              1.012e-04  3.092e-05   3.272 0.001123 ** 
BsmtFinSF2              5.885e-05  3.999e-05   1.471 0.141638    
BsmtUnfSF               3.480e-05  2.867e-05   1


Call:
lm(formula = sale_price_log ~ ., data = df.training2.train)

Residuals:
     Min       1Q   Median       3Q      Max 
-1.00213 -0.06121  0.00794  0.06605  0.40286 

Coefficients: (4 not defined because of singularities)
                         Estimate Std. Error t value Pr(>|t|)    
(Intercept)             4.335e+00  8.560e+00   0.506 0.612731    
LotFrontage             1.837e-04  1.676e-04   1.097 0.273218    
LotArea                 1.889e-01  2.063e-01   0.916 0.360245    
OverallQual             5.094e-02  7.159e-03   7.115 2.94e-12 ***
OverallCond             4.990e-02  6.411e-03   7.783 2.77e-14 ***
YearBuilt               6.466e-04  4.716e-04   1.371 0.170835    
YearRemodAdd            1.286e-04  4.074e-04   0.316 0.752334    
MasVnrArea             -1.408e-05  3.844e-05  -0.366 0.714330    
BsmtFinSF1             -1.551e-04  1.287e-04  -1.204 0.228837    
BsmtFinSF2             -7.922e-02  7.666e-02  -1.033 0.301828    
BsmtUnfSF              -2.154e-04  1.294e-04  -